In [8]:
# %matplotlib inline
# %pip install scikit-learn
import pandas as pd
import numpy as np  
import os
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import warnings
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from typing import Dict, List, Tuple, Optional, Union, Callable

# Suppress all warnings
warnings.filterwarnings('ignore')

# Configuration
SUPPLEMENT_FILE = 'supplementary_data.csv'
INPUT_PATTERN = 'input_2023_w{:02d}.csv'
OUTPUT_PATTERN = 'supplement-input_2023_w{:02d}.csv'
NUM_WEEKS = 18

In [9]:
# Load supplementary data
supplementary_data = pd.read_csv(os.path.join('data', SUPPLEMENT_FILE))

# Add feature pass_category based on pass_length. 
# pass_categories are 'short', 'intermediate', and 'long'. Short is <=5 yards, intermediate is >5 and <=15 yards, long is >15 yards. Use dict mapping.
conditions = [
    supplementary_data['pass_length'] < 5,
    (supplementary_data['pass_length'] >= 5) & (supplementary_data['pass_length'] <= 15),
    supplementary_data['pass_length'] > 15
]

categories = ['short', 'intermediate', 'long']

supplementary_data['pass_type'] = np.select(
    conditions, 
    categories, 
    default='unknown'
)

In [10]:
 # Concatenate all weekly data into one dataset
print(f"\n{'='*70}")
print("STEP 1: CONCATENATING ALL WEEKS INTO SINGLE DATASET")
print("="*70)

# Load all weeks data into one dataframe
input_data_frames = []
for week in range(1, NUM_WEEKS + 1):
    file_path = os.path.join('data', 'input', INPUT_PATTERN.format(week))
    df_week = pd.read_csv(file_path)
    df_week['week'] = week  # Add week column to track the source
    input_data_frames.append(df_week)

# Combine all dataframes
input_data = pd.concat(input_data_frames, ignore_index=True)
print(f"Loaded {len(input_data_frames)} weeks of data with {len(input_data)} total rows")
print(f"Test : {input_data['week'].max()}")


STEP 1: CONCATENATING ALL WEEKS INTO SINGLE DATASET
Loaded 18 weeks of data with 4880579 total rows
Test : 18


In [11]:
game = 2023090700
play = 101
# Get supplementary info
play_info = supplementary_data[
        (supplementary_data['game_id'] == game) & 
        (supplementary_data['play_id'] == play)]
display(play_info)

play_data = input_data[
        (input_data['game_id'] == game) & 
        (input_data['play_id'] == play)
    ].copy()

qb_data = play_data[play_data['player_role'] == 'Passer']
display(qb_data[['frame_id', 'x', 'y', 'dir']])

,game_id,season,week,game_date,game_time_eastern,home_team_abbr,visitor_team_abbr,play_id,play_description,quarter,...,penalty_yards,pre_penalty_yards_gained,yards_gained,expected_points,expected_points_added,pre_snap_home_team_win_probability,pre_snap_visitor_team_win_probability,home_team_win_probability_added,visitor_team_win_probility_added,pass_type
5,2023090700,2023,1,09/07/2023,20:20:00,KC,DET,101,(14:25) (Shotgun) J.Goff pass incomplete deep ...,1,...,NaN,0,0,0.927021,-2.145443,0.590426,0.409574,0.04972,-0.04972,long


,frame_id,x,y,dir
182,1,37.36,30.07,65.42
183,2,37.36,30.07,63.91
184,3,37.35,30.07,53.83
185,4,37.34,30.07,310.79
186,5,37.33,30.07,271.81
187,6,37.31,30.07,273.63
188,7,37.26,30.08,274.20
189,8,37.15,30.08,273.65
190,9,37.01,30.09,273.49
191,10,36.84,30.10,272.92


In [12]:
start_yard = play_info['yardline_number'].iloc[0] if not play_info.empty else None
print(type(start_yard))
start_yard = float(start_yard)
print(type(start_yard))
print(start_yard)


formation = play_info['offense_formation'].iloc[0] if not play_info.empty else 'Unknown'
route = play_info['route_of_targeted_receiver'].iloc[0] if not play_info.empty else 'Unknown'
start_yard = play_info['yardline_number'].iloc[0] if not play_info.empty else None
start_yard = float(start_yard)
yardline_side = play_info['yardline_side'].iloc[0] if not play_info.empty else 'Unknown'
possession_team = play_info['possession_team'].iloc[0] if not play_info.empty else 'Unknown'    
yards_to_go = play_info['yards_to_go'].iloc[0] if not play_info.empty else None
abs_yard_line = input_data[
    (input_data['game_id'] == game) & 
    (input_data['play_id'] == play)
]['absolute_yardline_number'].iloc[0] if not input_data.empty else None


#print(f"✅ Data found for Game {game_id}, Play {play_id}")
print(f"  - Formation: {formation}")
print(f"  - Route: {route}")
print(f"  - Line of Scrimmage: {start_yard} ({yardline_side})")
print(f"  - Possession Team: {possession_team}")

# Covert NLF start_yard to field coordinates i.e. LOC_X
# Determine field_start_x based on yardline_side and possession_team
# When yardline_side matches possession_team, LOC_X = 120 - start_yard
# Otherwise, LOC_X = start_yard
if yardline_side == possession_team:
    loc_x = 120 - start_yard
else:
    loc_x = start_yard
print(f"LOC_X: {loc_x}")
print(f"Yard to Go: {yards_to_go}")
print(f"Absolute Yard Line: {abs_yard_line}")

<class 'numpy.int64'>
<class 'float'>
32.0
  - Formation: SHOTGUN
  - Route: CORNER
  - Line of Scrimmage: 32.0 (DET)
  - Possession Team: DET
LOC_X: 88.0
Yard to Go: 3
Absolute Yard Line: 42


In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.animation import FuncAnimation, PillowWriter, FFMpegWriter
from typing import Optional, Literal
import warnings
warnings.filterwarnings('ignore')


def create_play_animation(
    tracking_df: pd.DataFrame,
    supplement_df: pd.DataFrame,
    game_id: int,
    play_id: int,
    output_path: str = 'play_animation',
    output_format: Literal['gif', 'mp4'] = 'gif',
    fps: int = 10,
    show_pressure_zones: bool = True,
    show_inline: bool = False,
    figsize: tuple = (16, 10)
) -> None:
    """
    Create animated visualization of NFL play with QB pressure zones and movement trails.
    
    Parameters
    ----------
    tracking_df : pd.DataFrame
        Tracking data with columns: game_id, play_id, frame_id, player_role, x, y, dir, player_name
    supplement_df : pd.DataFrame
        Supplementary data with columns: game_id, play_id, offense_formation, route_of_targeted_receiver, yardnumber
    game_id : int
        Game identifier
    play_id : int
        Play identifier
    output_path : str
        Output file path (without extension)
    output_format : {'gif', 'mp4'}
        Animation output format
    fps : int
        Frames per second
    show_pressure_zones : bool
        Whether to show QB pressure zones
    show_inline : bool
        Display animation inline (Jupyter)
    figsize : tuple
        Figure size (width, height)
    """
    
    # Filter data for specific play
    play_data = tracking_df[
        (tracking_df['game_id'] == game_id) & 
        (tracking_df['play_id'] == play_id)
    ].copy()
    
    if play_data.empty:
        print(f"❌ No data found for Game {game_id}, Play {play_id}")
        return
    
    # Get supplementary info
    play_info = supplement_df[
        (supplement_df['game_id'] == game_id) & 
        (supplement_df['play_id'] == play_id)
    ]
    
    formation = play_info['offense_formation'].iloc[0] if not play_info.empty else 'Unknown'
    route = play_info['route_of_targeted_receiver'].iloc[0] if not play_info.empty else 'Unknown'
    loc_x = tracking_df[
        (tracking_df['game_id'] == game_id) & 
        (tracking_df['play_id'] == play_id)
    ]['absolute_yardline_number'].iloc[0] if not tracking_df.empty else None  

    print(f"✅ Data found for Game {game_id}, Play {play_id}")
    print(f"  - Formation: {formation}")
    print(f"  - Route: {route}")
    print(f"  - Line of Scrimmage: {loc_x}")

    # Get frame range
    frames = sorted(play_data['frame_id'].unique())
    print(f"Creating animation with {len(frames)} frames...")
    print(f"Frame range: {frames[0]} to {frames[-1]}")
    
    # Separate player roles
    qb_data = play_data[play_data['player_role'] == 'Passer']
    receiver_data = play_data[play_data['player_role'] == 'Targeted Receiver']
    defender_data = play_data[play_data['player_role'] == 'Defensive Coverage']
    
    if qb_data.empty:
        print("❌ No QB found in play")
        return
    
    # Get field boundaries with padding
    all_x = play_data['x'].values
    all_y = play_data['y'].values
    print(f"All X range: {all_x.min()} to {all_x.max()}")
    print(f"All Y range: {all_y.min()} to {all_y.max()}")
    # x_min, x_max = all_x.min() - 10, all_x.max() + 10
    y_min, y_max = all_y.min() - 10, all_y.max() + 10
    
    # # Ensure start_yard is included in the range
    # loc_x is not none and find closed to min or max. Only add 8 yards padding to closest side. 
    if start_yard is not None:
        # Calculate padding based on distance to boundaries
        dist_to_min = abs(loc_x - all_x.min())
        dist_to_max = abs(all_x.max() - loc_x)
        
        padding_min = 8 if dist_to_min < dist_to_max else 2
        padding_max = 2 if dist_to_min < dist_to_max else 8
        
        x_min = min(all_x.min(), loc_x) - padding_min
        x_max = max(all_x.max(), loc_x) + padding_max
    
    
    # Initialize figure with white background   
    fig, ax = plt.subplots(figsize=figsize, facecolor='white')
    ax.set_facecolor('white')  # White background
    
    # Storage for trails
    qb_trail_x, qb_trail_y = [], []
    receiver_trail_x, receiver_trail_y = [], []
    
    def init():
        """Initialize animation"""
        ax.clear()
        setup_field(ax, x_min, x_max, y_min, y_max, loc_x)
        return []
    
    def update(frame_idx):
        """Update function for each frame"""
        ax.clear()
        setup_field(ax, x_min, x_max, y_min, y_max, loc_x)
        
        frame_id = frames[frame_idx]
        frame_data = play_data[play_data['frame_id'] == frame_id]
        
        # Get current positions
        qb_frame = frame_data[frame_data['player_role'] == 'Passer']
        receiver_frame = frame_data[frame_data['player_role'] == 'Targeted Receiver']
        defender_frame = frame_data[frame_data['player_role'] == 'Defensive Coverage']
        
        if not qb_frame.empty:
            qb_x, qb_y = qb_frame.iloc[0]['x'], qb_frame.iloc[0]['y']
            qb_dir = qb_frame.iloc[0]['dir']
            qb_name = qb_frame.iloc[0].get('player_name', 'QB')
            
            # Update QB trail
            qb_trail_x.append(qb_x)
            qb_trail_y.append(qb_y)
            
            # Draw QB trail
            if len(qb_trail_x) > 1:
                ax.plot(qb_trail_x, qb_trail_y, color='#1565c0', linewidth=3, 
                       alpha=0.6, linestyle='-', zorder=2)
            
            # Draw pressure zones
            if show_pressure_zones:
                zones = [
                    (7, '#ffeb3b', 0.15, 'Potential (7 yds)'),
                    (5, '#ff9800', 0.20, 'Closing (5 yds)'),
                    (3, '#f44336', 0.25, 'Immediate (3 yds)')
                ]
                for radius, color, alpha, label in zones:
                    circle = plt.Circle((qb_x, qb_y), radius, color=color, 
                                       alpha=alpha, fill=True, zorder=1, label=label)
                    ax.add_patch(circle)
            
            # Draw QB
            draw_player(ax, qb_x, qb_y, qb_dir, '#1565c0', qb_name, size=600, zorder=5)
        
        # Draw receiver
        if not receiver_frame.empty:
            rec_x, rec_y = receiver_frame.iloc[0]['x'], receiver_frame.iloc[0]['y']
            rec_dir = receiver_frame.iloc[0]['dir']
            rec_name = receiver_frame.iloc[0].get('player_name', 'WR')
            
            # Update receiver trail
            receiver_trail_x.append(rec_x)
            receiver_trail_y.append(rec_y)
            
            # Draw receiver trail
            if len(receiver_trail_x) > 1:
                ax.plot(receiver_trail_x, receiver_trail_y, color='#64b5f6', 
                       linewidth=2.5, alpha=0.6, linestyle='-', zorder=2)
            
            # Draw receiver
            draw_player(ax, rec_x, rec_y, rec_dir, '#64b5f6', rec_name, size=450, zorder=5)
        
        # Draw defenders
        if not qb_frame.empty:
            qb_x, qb_y = qb_frame.iloc[0]['x'], qb_frame.iloc[0]['y']
            
            for _, defender in defender_frame.iterrows():
                def_x, def_y = defender['x'], defender['y']
                def_dir = defender['dir']
                def_name = defender.get('player_name', 'D')
                
                # Calculate distance to QB
                distance = np.sqrt((def_x - qb_x)**2 + (def_y - qb_y)**2)
                
                # Color by distance with more visible contrast
                if distance < 3:
                    color = '#8B0000'  # Dark red (immediate threat)
                    edge_color = 'yellow'
                    edge_width = 2.5
                elif distance < 5:
                    color = '#DC143C'  # Crimson (closing threat)
                    edge_color = 'white'
                    edge_width = 2.0
                elif distance < 7:
                    color = '#FF4500'  # Orange red (potential threat)
                    edge_color = 'white'
                    edge_width = 1.5
                else:
                    color = '#FF6347'  # Tomato (distant)
                    edge_color = 'white'
                    edge_width = 1.0
                
                # Draw defender with visible edge
                draw_player(ax, def_x, def_y, def_dir, color, '', size=350, 
                          zorder=4, edge_color=edge_color, edge_width=edge_width)
                
                # Add distance label with white background for visibility
                ax.text(def_x, def_y + 1.2, f'{distance:.1f}', 
                       fontsize=9, ha='center', color='black', fontweight='bold',
                       bbox=dict(boxstyle='round,pad=0.2', facecolor='white', alpha=0.7))
                    #    bbox=dict(boxstyle='round,pad=0.3', facecolor='white', 
                    #             edgecolor='black', linewidth=1.5, alpha=0.85))
        
        # Add legend
        from matplotlib.patches import Patch
        from matplotlib.lines import Line2D
        
        legend_elements = [
            Line2D([0], [0], marker='^', color='w', markerfacecolor='#1565c0', 
                   markersize=12, label='Quarterback', markeredgecolor='white', markeredgewidth=1.5),
            Line2D([0], [0], marker='^', color='w', markerfacecolor='#64b5f6', 
                   markersize=10, label='Receiver', markeredgecolor='white', markeredgewidth=1.5),
            Patch(facecolor='#f44336', alpha=0.25, label='Immediate Threat (3 yds)'),
            Patch(facecolor='#ff9800', alpha=0.20, label='Closing Threat (5 yds)'),
            Patch(facecolor='#ffeb3b', alpha=0.15, label='Potential Threat (7 yds)'),
        ]
              
        # Place legend
        ax.legend(handles=legend_elements, loc='upper right', fontsize=9,
                 framealpha=0.9, facecolor='white', edgecolor='black',
                 labelcolor='black', title='Legend', title_fontsize=10)
        
        # Title with play info
        ax.set_title(f'Game {game_id} - Play {play_id} | Frame {frame_id} \n Formation: {formation} | Route: {route}',
                     fontsize=16, color='black', fontweight='bold', pad=20)
        
        return []
    
    # Create animation
    anim = FuncAnimation(fig, update, init_func=init, frames=len(frames), 
                        interval=1000/fps, blit=True, repeat=True)
    
    # Save animation
    output_file = f"{output_path}.{output_format}"
    print(f"Saving animation to {output_file}...")
    
    if output_format == 'gif':
        writer = PillowWriter(fps=fps)
        anim.save(output_file, writer=writer, dpi=100)
    else:  # mp4
        writer = FFMpegWriter(fps=fps, bitrate=1800)
        anim.save(output_file, writer=writer, dpi=100)
    
    print(f"✅ Animation saved successfully!")
    print(f"  - Frames: {len(frames)}")
    print(f"  - FPS: {fps}")
    print(f"  - Duration: {len(frames)/fps:.1f} seconds")
    print(f"  - Output: {output_file}")
    
    if not show_inline:
        plt.close()


def setup_field(ax, x_min, x_max, y_min, y_max, loc_x=None):
    """Setup NFL field styling with yard lines"""
    
    # Set limits
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)
    
    # Draw yard lines (every 5 yards)
    yard_start = int(x_min // 5) * 5
    yard_end = int(x_max // 5 + 1) * 5
    
    for yard in range(yard_start, yard_end, 5):
        if x_min <= yard <= x_max:
            ax.axvline(yard, color='black', alpha=0.3, linewidth=1, linestyle='-', zorder=0)

    if loc_x is not None and x_min <= loc_x <= x_max:
        ax.axvline(loc_x, color='black', linewidth=3, linestyle='-', zorder=1)
        ax.text(loc_x, y_min + 1, 'LOS', color='green', fontsize=10, 
               ha='center', fontweight='bold',
               bbox=dict(boxstyle='round,pad=0.3', facecolor='black', alpha=0.8))
    
    # Sidelines
    ax.axhline(y_min, color='black', linewidth=2, zorder=0)
    ax.axhline(y_max, color='black', linewidth=2, zorder=0)
    
    # Labels
    ax.set_xlabel('Field X Position (yards)', fontsize=11, color='black', fontweight='bold')
    ax.set_ylabel('Field Y Position (yards)', fontsize=11, color='black', fontweight='bold')
    
    # Tick styling
    ax.tick_params(colors='black', labelsize=9)
    ax.grid(False)
    ax.set_aspect('equal')


def draw_player(ax, x, y, direction, color, name, size=400, zorder=3, 
                edge_color='white', edge_width=1.5):
    """
    Draw player as triangle oriented by movement direction
    
    Parameters
    ----------
    ax : matplotlib axis
    x, y : float
        Player position
    direction : float
        Direction angle in degrees (0-360)
    color : str
        Player color
    name : str
        Player name
    size : int
        Marker size
    zorder : int
        Drawing layer
    edge_color : str
        Edge color for visibility
    edge_width : float
        Edge line width
    """
    
    # Convert direction to radians and adjust for matplotlib
    # NFL direction: 0=North, 90=East, 180=South, 270=West
    # Matplotlib rotation: counter-clockwise from East
    angle_rad = np.radians(90 - direction)
    
    # Create triangle vertices (pointing up initially)
    triangle_size = np.sqrt(size) / 15
    vertices = np.array([
        [0, triangle_size],      # Top point
        [-triangle_size/2, -triangle_size/2],  # Bottom left
        [triangle_size/2, -triangle_size/2]    # Bottom right
    ])
    
    # Rotate vertices
    rotation_matrix = np.array([
        [np.cos(angle_rad), -np.sin(angle_rad)],
        [np.sin(angle_rad), np.cos(angle_rad)]
    ])
    rotated = vertices @ rotation_matrix.T
    
    # Translate to position
    rotated[:, 0] += x
    rotated[:, 1] += y
    
    # Draw triangle
    triangle = patches.Polygon(rotated, closed=True, facecolor=color, 
                              edgecolor=edge_color, linewidth=edge_width, zorder=zorder)
    ax.add_patch(triangle)
    
    # Add name label if provided
    if name:
        ax.text(x, y - 1.5, name, fontsize=8, ha='center', color='white',
               fontweight='bold',
               bbox=dict(boxstyle='round,pad=0.2', facecolor=color, alpha=0.9))




In [15]:
# =============================================================================
# EXAMPLE USAGE
# =============================================================================

if __name__ == "__main__":
    """
    Example usage with your data structure
    
    Required columns in tracking_df:
    - game_id, play_id, frame_id
    - player_role: 'Passer', 'Targeted Receiver', 'Defensive Coverage'
    - x, y: field coordinates
    - dir: direction angle (0-360 degrees)
    - player_name: (optional)
    
    Required columns in supplement_df:
    - game_id, play_id
    - offense_formation
    - route_of_targeted_receiver
    - yardnumber: line of scrimmage
    """
    game = 2023111210 #2023111911 #2023092403 #2023111203 #2023091005 #2023090700 #2023091008 #2023091007 #2023091012 #2023090700 #2023091100
    play = 3123 #3724 #1704 #221 #2440 #2186  #699 #3298 # 101 # 3214
    # Example call (uncomment and modify with your data):
    create_play_animation(
        tracking_df=input_data,
        supplement_df=supplementary_data,
        game_id=game,
        play_id=play,
        output_path=f'Game-{game}_PlayID-{play}',
        output_format='gif',
        fps=10,
        show_pressure_zones=True,
        show_inline=False
    )
    
    print("Animation function ready!")
    print("\nUsage:")
    print("create_play_animation(tracking_df, supplement_df, game_id, play_id, 'output_path')")

✅ Data found for Game 2023111210, Play 3123
  - Formation: SHOTGUN
  - Route: HITCH
  - Line of Scrimmage: 79
Creating animation with 48 frames...
Frame range: 1 to 48
All X range: 59.73 to 87.76
All Y range: 5.53 to 48.03
Saving animation to Game-2023111210_PlayID-3123.gif...
✅ Animation saved successfully!
  - Frames: 48
  - FPS: 10
  - Duration: 4.8 seconds
  - Output: Game-2023111210_PlayID-3123.gif
Animation function ready!

Usage:
create_play_animation(tracking_df, supplement_df, game_id, play_id, 'output_path')
